In [1]:
import Pkg;
Pkg.add("JuMP")
Pkg.add("HiGHS")
Pkg.add("Graphs")

┌ Warning: could not download https://pkg.julialang.org/registries
│   exception = Downloads.RequestError("https://pkg.julialang.org/registries", 6, "Could not resolve host: pkg.julialang.org", Downloads.Response(nothing, "https://pkg.julialang.org/registries", 0, "", Pair{String, String}[]))
└ @ Pkg.Registry /Users/julia/.julia/scratchspaces/a66863c6-20e8-4ff4-8a62-49f30b1f605e/agent-cache/default-macmini-x64-6.0/build/default-macmini-x64-6-0/julialang/julia-release-1-dot-8/usr/share/julia/stdlib/v1.8/Pkg/src/Registry/Registry.jl:68
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Chan

In [2]:
using JuMP
using HiGHS
using Graphs

In [5]:
struct Clients
    nb_clients
    X
    Y
    h
    L
    L0
    d
end

In [3]:
function Read_PRP_instance(filename)
    hashmap = Dict()
    open(filename) do f
        
        demands = false
        listing_active = false
        current_value = 0
        
        nbc = 0
        X = Array{Float64}(undef, 0)
        Y = Array{Float64}(undef, 0)
        h = Array{Float64}(undef, 0)
        L = Array{Float64}(undef, 0)
        L0 = Array{Float64}(undef, 0)
        d = Array{Array{Float64}}(undef, 0)
        
        for (i, line) in enumerate(eachline(f))
            x = split(line, " ")
            deleteat!(x, findall(e->e=="", x))
            current_item = x[1]
            
            if(current_item == "0" || listing_active) # Liste de clients / demandes
                listing_active = true
                if(current_item == "d")
                    demands = true
                elseif(demands) # Enregistrement des demandes
                    push!(d, parse.(Float64, x))
                else # enregistrement des clients
                    nbc += 1
                    push!(X, parse(Float64, x[2]))
                    push!(Y, parse(Float64, x[3]))
                    push!(h, parse(Float64, x[6]))
                    push!(L, parse(Float64, x[8]))
                    push!(L0, parse(Float64, x[10]))
                end
            else # variables générales du problème
                try
                    current_value = parse(Int64, x[2])
                catch err
                    current_value = 1^10
                end
                hashmap[current_item] = current_value
            end
        end
        C = Clients(nbc, X, Y, h, L, L0, d)
        hashmap["Clients"] = C
    end 
    return hashmap
end

Read_PRP_instance (generic function with 1 method)

In [6]:
h = Read_PRP_instance("./PRP_instances/A_014_#ABS1_15_1.prp")

Dict{Any, Any} with 9 entries:
  "f"       => 3000
  "Q"       => 322
  "C"       => 1
  "k"       => 2085
  "Type"    => 1
  "u"       => 30
  "l"       => 6
  "Clients" => Clients(15, [143.0, 89.0, 76.0, 285.0, 401.0, 16.0, 267.0, 249.0…
  "n"       => 14

In [62]:
h["Clients"].d

14-element Vector{Array{Float64}}:
 [1.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0]
 [2.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0]
 [3.0, 15.0, 15.0, 15.0, 15.0, 15.0, 15.0]
 [4.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0]
 [5.0, 13.0, 13.0, 13.0, 13.0, 13.0, 13.0]
 [6.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0]
 [7.0, 22.0, 22.0, 22.0, 22.0, 22.0, 22.0]
 [8.0, 13.0, 13.0, 13.0, 13.0, 13.0, 13.0]
 [9.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0]
 [10.0, 22.0, 22.0, 22.0, 22.0, 22.0, 22.0]
 [11.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0]
 [12.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0]
 [13.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0]
 [14.0, 19.0, 19.0, 19.0, 19.0, 19.0, 19.0]

In [8]:
function dist(C, i, j, t, mc)
    if(t == 1)
        return ((C.X[i] - C.X[j])^2 + (C.Y[i] - C.Y[j])^2)^(0.5) + 0.5
    else
        return ((C.X[i] - C.X[j])^2 + (C.Y[i] - C.Y[j])^2)^(0.5) * mc
    end
end

dist (generic function with 1 method)

In [9]:
function calcul_dist(p)
    if(p["Type"] == 1)
        mc = 0
    else
        mc = p["mc"]
    end
    c = Array{Float64}(undef, (p["n"]+1, p["n"]+1))
    for i in 1:p["n"]+1
        for j in 1:p["n"]+1            
            c[i, j] = dist(p["Clients"], i, j, p["Type"], mc)
        end
    end
    return c
end

calcul_dist (generic function with 1 method)

In [10]:
function init_SC(c,n,l)
    SC = Array{Array{Float64}}(undef, (n,l))
    for i in 1:n+1
        for t in 1:l+1            
            SC[i, t] = c[1,i] + c[i,1] 
        end
    end
    return SC
end

init_SC (generic function with 1 method)

In [11]:
sum(h["Clients"].h[i] for i in 1:1)

3.0

In [12]:
function Resolve_LSP(p)
    LP = Model(HiGHS.Optimizer)
    
    c = calcul_dist(p)    
    n = p["Clients"].nb_clients
    L0 = p["Clients"].L0
    L = p["Clients"].L
    l = p["l"]+1
    M = 10000
    SC = init_SC(c,n,l)

    
    @variable(LP, y[1:l], Bin)  # Production démarrée
    @variable(LP, prod[1:l], Bin)  # Quantité produite
    @variable(LP, inv[1:n, 1:l], Int)  # Inventaire des clients
    @variable(LP, x[1:n, 1:n, 1:l], Bin)  # Camion entre i et j au temps t
    @variable(LP, q[1:n, 1:l], Int)
    @variable(LP, z[1:n, 1:l], Bin)  # zit vaut 1 si le client i est visité à la période t

    
    @objective(LP, Min, sum(p["u"]*prod[t] + p["f"]*y[t] + 
            sum(p["Clients"].h[i]*inv[i, t] for i in 1:n) + 
            sum(sum(c[i, j] * x[i, j, t] for i in 1:n) for j in 1:n) for t in 1:l) + 
            sum(sum(SC[i,t] * z[i,t] for i in 1:n) for t in 1:l)) 
    
    for t in 1:l
        if(t ≠ 1)
            @constraint(LP, inv[1, t-1] + prod[t] == sum(q[i, t] for i in 1:n) + inv[1, t]) # (1) OK
            @constraint(LP,inv[1, t-1] <= L0) #(4)
        end
        for i in 1:n
            if(i ≠ 1 && t ≠ 1)
                @constraint(LP, inv[i, t-1] + q[i, t] == p["Clients"].d[i-1][t] + inv[i, t]) # (2) OK
                @constraint(LP, inv[i, t-1] + q[i, t] <= L[i]) # (5)
                
                @constraint(LP,sum(x[i,j,t] for j in 2:n) == z[i,t]) # (1 bis) un arc vers i est emprunté au moins une fois au temps t si un véhicule visite i au temps t
                @constraint(LP,sum(x[i,j,t] for j in 2:n) + sum(x[j,i,t] for j in 2:n) == 2 * z[i,t]) # (2 bis) conservation de flot des véhicules
                @constraint(LP,q[i,t] <= M*z[i,t]) # (3 bis) la quantité livrée en t est positive si i a été livrée en t
            end
            
        end
        @constraint(LP,prod[t] <= M*y[t]) # (3) M est une variable arbitrairement grande
        
    end
    for i in 1:n
            if(i ≠ 1)
                @constraint(LP, sum(z[i, t] for t in 2:l) >= 1) #on visite un client au moins une fois
            end
        end
    
    # print(LP)
    println()

    #optimize!(LP)
   
    #println(solution_summary(LP, verbose=true))
    
    return 0
end

Resolve_LSP (generic function with 1 method)

In [13]:
Resolve_LSP(Read_PRP_instance("./PRP_instances/A_014_#ABS1_15_1.prp"))

LoadError: MethodError: [0mCannot `convert` an object of type 
[0m  [92mFloat64[39m[0m to an object of type 
[0m  [91mArray{Float64}[39m
[0mClosest candidates are:
[0m  convert(::Type{T}, [91m::LinearAlgebra.Factorization[39m) where T<:AbstractArray at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/LinearAlgebra/src/factorization.jl:58
[0m  convert(::Type{Array{T}}, [91m::StaticArraysCore.SizedArray{S, T, N, M, Array{T, M}}[39m) where {S, T, N, M} at ~/.julia/packages/StaticArrays/PUoe1/src/SizedArray.jl:79
[0m  convert(::Type{Array{T}}, [91m::StaticArraysCore.SizedArray{S, T, N, M, TData} where {N, M, TData<:AbstractArray{T, M}}[39m) where {T, S} at ~/.julia/packages/StaticArrays/PUoe1/src/SizedArray.jl:76
[0m  ...

In [14]:
Resolve_LSP(Read_PRP_instance("./PRP_instances/B_100_instance17.prp"))

LoadError: MethodError: [0mCannot `convert` an object of type 
[0m  [92mFloat64[39m[0m to an object of type 
[0m  [91mArray{Float64}[39m
[0mClosest candidates are:
[0m  convert(::Type{T}, [91m::LinearAlgebra.Factorization[39m) where T<:AbstractArray at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/LinearAlgebra/src/factorization.jl:58
[0m  convert(::Type{Array{T}}, [91m::StaticArraysCore.SizedArray{S, T, N, M, Array{T, M}}[39m) where {S, T, N, M} at ~/.julia/packages/StaticArrays/PUoe1/src/SizedArray.jl:79
[0m  convert(::Type{Array{T}}, [91m::StaticArraysCore.SizedArray{S, T, N, M, TData} where {N, M, TData<:AbstractArray{T, M}}[39m) where {T, S} at ~/.julia/packages/StaticArrays/PUoe1/src/SizedArray.jl:76
[0m  ...

In [ ]:
Read_PRP_instance("./PRP_instances/B_100_instance17.prp")

In [80]:
function clark_wright(params,demands,costs,t)

    n = params["n"] #n est le nombre de clients
    Q = params["Q"] #Q est la capacité des camions


    #Initialiser S
    S = [[0, i] for i in 1:n]
    whereIs = Dict(i => circuit for (i, circuit) in enumerate(S))

    #Calcul des s_{i,j} et mettre dans l'ordre décroissant
    s=[]
    for i in 1:n, j in 1:n
        if i != j
            push!(s, [ (i,j), costs[1, i+1] + costs[1, j+1] - costs[i+1, j+1] ]) # pb indices
        end
    end
    sort!(s, by = x->x[2], rev=true) #by = key pour sort, rev = reverse (ordre décroissant)

    #Construction des circuits (attention il peut en avoir plus de m (le nombre de camion disponible))
    for k in 1:length(s)

        i=s[k][1][1]
        j=s[k][1][2]

        circuit_i = whereIs[i]
        circuit_j = whereIs[j]

        #Si i et j ne sont pas dans le même circuit
        if circuit_j != circuit_i

            index_circuit_i = findfirst(item -> item == circuit_i, S)
            index_circuit_j = findfirst(item -> item == circuit_j, S)

            circuit_union = union(S[index_circuit_i], S[index_circuit_j])

            #Calcul demande du nouveau circuit circuit_union
            demandeUnion=0
            for l in circuit_union[2:end] #commence à 2 car le premier noeud du circuit est 0 (le dépot n'a pas de demande)
                demandeUnion += demands[l][t]
            end
            
            #Si la demande du nouveau circuit n'est pas trop pour un seul camion
            if demandeUnion <= Q
                deleteat!(S, sort!([index_circuit_i, index_circuit_j]))

                push!(S, circuit_union)


                for index in circuit_union
                    push!(whereIs, index=>circuit_union)
                end

            end
        end

    end

    return S #la liste des circuits qui sont des tournées valides (qui respectent la contrainte de poids)
end

clark_wright (generic function with 2 methods)

In [81]:
costs = calcul_dist(Read_PRP_instance("./PRP_instances/B_100_instance17.prp"))
demands = h["Clients"].d
clark_wright(h,demands,costs,6)

3-element Vector{Vector{Int64}}:
 [0, 1, 4, 14, 2, 7, 13]
 [0, 3, 6, 10, 12, 11]
 [0, 8, 5, 9]